In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

from shapely.geometry import shape, Point, Polygon
from shapely.ops import unary_union

import rtree as rt

%matplotlib inline
pd.set_option("display.max.columns", None)

## TODOs:
- Based on call with Mo: weighted average of three nearest sensors, figure out AQI, use weekly average data
- Retitle columns to be useful
- Find more complete census tract geojson/one that isn't hosted by latimes is liable to go away at any moment
- How to handle tracts with multiple sensors in them?
- Determine flow:
    - Census tracts won't change, but each sensor can go offline
    - How often to collect PurpleAir data
        - Entire pipeline needs to run at each collection to ensure current/accurate sensors
    - Census tract polygon object gets mapped to a nearest PurpleAir sensor
        - How to make nearest neighbor calculation as efficient as possible
    - How to rebuild geojson/arcgis object from dataframes
    - Where to host processed data?
    - How to package data for dashboard
- Schedule hosted feature layer to be updated:
    - https://learn.arcgis.com/en/projects/schedule-automated-near-real-time-data-updates/

### PurpleAir Data descriptors
##### Determined
- pm = current PM2.5 reading
- pm1 = raw PM1 reading
- pm_10 = raw PM10 reading
- pm_0 = current PM2.5 reading
- pm_1 = 10 minute PM2.5 average
- pm_2 = 30 minute PM2.5 average
- pm_3 = 1 hour PM2.5 average
- pm_4 = 6 hour PM2.5 average
- pm_5 = 24 hour PM2.5 average
- pm_6 = One week PM2.5 average
- p1 = Particles >= 0.3 µm
- p2 = Particles >= 0.5 µm
- p3 = Particles >= 1.0 µm
- p4 = Particles >= 2.5 µm
- p5 = Particles >= 5.0 µm
- p6 = Particles >= 10.0 µm
- flags = Data flagged for unusually high readings
- age = Sensor data age (when data was last received) in minutes
- isOwner = Currently logged in user is the sensor owner
- Adc = The voltage reading on the analog input of the control board

##### Undetermined
- conf
- Icon
- Voc
- Ozone1
- CH

In [23]:
# Returns dataframe of active purpleair sensors contained within LA County
def los_angeles_county_sensors(sensors_df):
    # Set boolean to use LAC shape or LAC bounding box (includes some Orange County and Ventura County)
    use_shape = False
    
    # Initialize rtree index
    idx = rt.index.Index()
    
    # Get LA County geometries in JSON form
    url = 'https://opendata.arcgis.com/datasets/10f1e37c065347e693cf4e8ee753c09b_15.geojson'
    shapes = requests.get(url).json()
    
    la_county_indeces = []
    
    # Union of individual LA County shapes. Speeds up check if sensor is in LAC.
    polygons = [shape(feature['geometry']) for feature in shapes['features']]
    lac_shape = unary_union(polygons)
    lac_bounds = lac_shape.bounds
    
    
    for index, row in sensors_df.iterrows():
        if ~np.isnan(row.Lon) and ~np.isnan(row.Lat):
            
            if use_shape:
                # Build Point object from sensor's Lon and Lat (in that order!) values
                point = Point(row.Lon, row.Lat)

                # Populate rtree index with point bounds
                idx.insert(int(index), point.bounds, point)
            else: 
                if lac_bounds[0] <= row.Lon and row.Lon <= lac_bounds[2] and lac_bounds[1] <= row.Lat and row.Lat <= lac_bounds[3]:
                    la_county_indeces.append(int(index))
    
    if use_shape:
        # get list of fids where bounding boxes intersect
        fids = [i for i in idx.intersection(lac_shape.bounds, objects=True)]
        print(fids[0].object)

        # access the features that those fids reference
        for fid in fids:
            # check the geometries intersect, not just their bboxs
            if lac_shape.contains(fid.object):
                la_county_indeces.append(fid.id)
                    
                
    return sensors_df.loc[la_county_indeces]

In [24]:
# Use 'experimental' data from purpleair
url = 'https://www.purpleair.com/data.json'
data = requests.get(url).json()

sensors_df = pd.DataFrame(data['data'], columns=data['fields'])
sensors_df = sensors_df.set_index('ID')

# Currently only using 1 week PM2.5 average ('pm_6')
cols_to_keep = ['pm_6', 'Lat', 'Lon']
cols_to_drop = [col for col in sensors_df.columns if col not in cols_to_keep]

sensors_df = sensors_df.drop(cols_to_drop, axis=1)

In [25]:
# Get purpleair sensors in LA County
la_county_sensors_df = los_angeles_county_sensors(sensors_df)
la_county_sensors_df['point'] = la_county_sensors_df.apply(lambda x: Point(x['Lon'], x['Lat']), axis=1)
la_county_sensors_df

,pm_6,Lat,Lon,point
ID,,,,
185,27.8,34.133865,-118.233530,POINT (-118.23353 34.133865)
407,27.9,33.842610,-118.339360,POINT (-118.33936 33.84261)
417,26.3,33.716675,-118.309906,POINT (-118.309906 33.716675)
489,29.4,33.795550,-118.260940,POINT (-118.26094 33.79555)
565,16.6,33.709614,-118.287970,POINT (-118.28797 33.709614)
...,...,...,...,...
74317,1.9,34.036250,-118.427760,POINT (-118.42776 34.03625)
74417,3.0,34.434200,-118.482796,POINT (-118.482796 34.4342)
74693,2.2,34.020546,-118.361640,POINT (-118.36164 34.020546)


In [26]:
# Populate rtree index with sensor points
idx = rt.index.Index()
for index, row in la_county_sensors_df.iterrows():
    idx.insert(int(index), row['point'].bounds)

#### Census Tract data is provided by LATimes (not ideal), but is complete.

In [27]:
# Read from url
# test_url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/census-tracts-2012.geojson'
# shape_data = requests.get(test_url).json()

# Open saved file
with open('census-tracts-2012.geojson') as f:
    shape_data = json.load(f)

In [28]:
census_tract_df = pd.DataFrame()

census_tract_df['tract'] = [int(feature['properties']['name']) for feature in shape_data['features']]
census_tract_df['coordinates'] = [feature['geometry']['coordinates'] for feature in shape_data['features']]
census_tract_df['shape'] = [shape(feature['geometry']) for feature in shape_data['features']]
census_tract_df.set_index('tract', inplace=True)
census_tract_df

,coordinates,shape
tract,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ..."
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ..."
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ..."
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ..."
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ..."
...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ..."
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ..."
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ..."


In [29]:
# Use rtree index's nearest function to find indexed point closest to the bounding box of the census tract
# Could easily get the n closest sensors to calculate an average
nearest_sensors = []
for index, row in census_tract_df.iterrows():
    geometry = row['shape']
    
    nearest = list(idx.nearest((geometry.bounds), 3))[:3]
    nearest_sensors.append(nearest)
census_tract_df['nearest_sensor_indices'] = nearest_sensors
census_tract_df

,coordinates,shape,nearest_sensor_indices
tract,,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ...","[71813, 67111, 6420]"
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ...","[71813, 67111, 64765]"
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ...","[67111, 71813, 6420]"
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ...","[67111, 63387, 71813]"
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ...","[67111, 63387, 23199]"
...,...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ...","[565, 4417, 1974]"
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ...","[22317, 9106, 57517]"
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ...","[26149, 19991, 38393]"


In [30]:
pm_6_cols = []
for index, row in census_tract_df.iterrows():
    closest_vals = []
    
    for sensor_index in row['nearest_sensor_indices']:
        sensor = la_county_sensors_df.loc[sensor_index]
        closest_vals.append(sensor['pm_6'])
    
    pm_6_cols.append(closest_vals)
census_tract_df['pm_6_vals'] = pm_6_cols
census_tract_df

,coordinates,shape,nearest_sensor_indices,pm_6_vals
tract,,,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ...","[71813, 67111, 6420]","[12.7, 20.2, 28.7]"
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ...","[71813, 67111, 64765]","[12.7, 20.2, 34.9]"
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ...","[67111, 71813, 6420]","[20.2, 12.7, 28.7]"
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ...","[67111, 63387, 71813]","[20.2, 40.2, 12.7]"
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ...","[67111, 63387, 23199]","[20.2, 40.2, 3.9]"
...,...,...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ...","[565, 4417, 1974]","[16.6, 27.2, 28.2]"
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ...","[22317, 9106, 57517]","[29.5, 31.3, 31.6]"
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ...","[26149, 19991, 38393]","[12.1, 29.3, 28.1]"


In [31]:
# Calculate weighted average
census_pm_6 = []

for index, row in census_tract_df.iterrows():
    distances = []
    values = []
    polygon = row['shape']
    for sensor_index in row['nearest_sensor_indices']:
        sensor = la_county_sensors_df.loc[sensor_index]
        distances.append(polygon.boundary.distance(sensor.point))
        values.append(sensor['pm_6'])
    
    weighted_avg = np.average(values, weights=distances)
    
    census_pm_6.append(weighted_avg)
    
census_tract_df['avg_pm_6'] = census_pm_6
census_tract_df

,coordinates,shape,nearest_sensor_indices,pm_6_vals,avg_pm_6
tract,,,,,
6037101110,"[[[[-118.302291, 34.258697], [-118.300787, 34....","(POLYGON ((-118.302291 34.258697, -118.300787 ...","[71813, 67111, 6420]","[12.7, 20.2, 28.7]",21.731279
6037101122,"[[[[-118.303334, 34.273536], [-118.303178, 34....","(POLYGON ((-118.303334 34.273536, -118.303178 ...","[71813, 67111, 64765]","[12.7, 20.2, 34.9]",29.293347
6037101210,"[[[[-118.299451, 34.255978], [-118.285924, 34....","(POLYGON ((-118.299451 34.255978, -118.285924 ...","[67111, 71813, 6420]","[20.2, 12.7, 28.7]",20.910351
6037101220,"[[[[-118.285924, 34.248959], [-118.285924, 34....","(POLYGON ((-118.285924 34.248959, -118.285924 ...","[67111, 63387, 71813]","[20.2, 40.2, 12.7]",22.412939
6037101300,"[[[[-118.272473, 34.232527], [-118.271936, 34....","(POLYGON ((-118.272473 34.232527, -118.271936 ...","[67111, 63387, 23199]","[20.2, 40.2, 3.9]",9.688411
...,...,...,...,...,...
6037980031,"[[[[-118.285303, 33.708598], [-118.283369, 33....","(POLYGON ((-118.285303 33.708598, -118.283369 ...","[565, 4417, 1974]","[16.6, 27.2, 28.2]",25.541977
6037980033,"[[[[-118.244627, 33.710767], [-118.231803, 33....","(POLYGON ((-118.244627 33.710767, -118.231803 ...","[22317, 9106, 57517]","[29.5, 31.3, 31.6]",31.088458
6037990100,"[[[[-118.951142, 33.996432], [-118.950564, 34....","(POLYGON ((-118.951142 33.996432, -118.950564 ...","[26149, 19991, 38393]","[12.1, 29.3, 28.1]",22.028783


In [ ]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [40]:
geojson = {'type':'FeatureCollection', 'features':[]}
for tract, row in census_tract_df.iterrows():
    feature = {'type': 'Feature',
              'properties': {},
              'geometry': {'type': 'MultiPolygon',
                          'coordinates': []}}
    feature['geometry']['coordinates'] = row['coordinates']
    feature['properties']['tract'] = '0' + str(tract)
    feature['properties']['pm_6_weighted_avg'] = row['avg_pm_6']
    geojson['features'].append(feature)
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'tract': '06037101110',
    'pm_6_weighted_avg': 21.731279171192433},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-118.302291, 34.258697],
       [-118.300787, 34.258773],
       [-118.300803, 34.263274],
       [-118.294338, 34.2632],
       [-118.294192, 34.262886],
       [-118.286486, 34.262797],
       [-118.284913, 34.262463],
       [-118.28498, 34.255889],
       [-118.299451, 34.255978],
       [-118.302291, 34.258697]]]]}},
  {'type': 'Feature',
   'properties': {'tract': '06037101122',
    'pm_6_weighted_avg': 29.29334717326267},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-118.303334, 34.273536],
       [-118.303178, 34.273956],
       [-118.302164, 34.274158],
       [-118.300959, 34.275032],
       [-118.299696, 34.275262],
       [-118.299357, 34.276027],
       [-118.299749, 34.276706],
       [-118.297835, 34.277791],
       [-118.297323, 34.278237],


In [50]:
output_filename = 'data.geojson'
with open(output_filename, 'a') as output_file:
#     output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 


# with open('data.geojson', 'a') as f:
#     json.dump(geojson, f)

In [51]:
len(geojson['features'])

2346

#### Census Tract data is provided by LATimes (not ideal), but is complete.

In [10]:
# # Iterate through sensors, checking which shape object the sensor's lat/lon falls within
# ind_cen = {}
# for index, row in la_county_sensors_df.iterrows():
#     # Build Point object from sensor's Lon and Lat (in that order!) values
#     point = Point(row.Lon, row.Lat)
    
#     for tract, polygon in polygons.items():
#         if polygon.contains(point):
#             ind_cen[index] = tract

In [11]:
# [i for i in la_county_sensors_df.index if i not in ind_cen.keys()]

#### Different approach by querying fcc's database to get census tract
#### 3.5 minutes to query for 560 locations

In [12]:
# # 3.5 minutes to query for 560 locations
# %%time
# tracts = {}
# for index, row in la_county_sensors_df.iterrows():
#     url = f'https://geo.fcc.gov/api/census/area?lat={row.Lat}&lon={row.Lon}&format=json'
#     results = requests.get(url).json()
#     tracts[index] = results['results'][0]['block_fips'][:11]

In [13]:
# for index, census in ind_cen.items():
#     if census != tracts[index]:
#         print('LATIMES: ', ind_cen[index], 'QUERY: ', tracts[index])